In [13]:
import gradio as gr
from langchain.memory import ConversationBufferMemory
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
#from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.llms import Ollama
from get_upload_func import save_uploaded_file

In [ ]:
import os
import pymupdf
from langchain.docstore.document import Document
from langchain_chroma import Chroma

def save_uploaded_file(pdf_file, embed_method, db_name="vector-db", uploaded_files_dir="uploaded_files"):
    """
    Yüklenen PDF dosyasını belirtilen klasöre kaydeder, metni işler ve vektör veritabanına ekler.
    
    :param pdf_file: gradio tarafından yüklenen dosya objesi
    :param embed_method: Seçilen embedding yöntemi
    :param db_name: Vektör veritabanının adı (varsayılan: "vector-db")
    :param uploaded_files_dir: Kaydedilecek ana dizin (varsayılan: "uploaded_files")
    """
    try:
        # Klasör oluştur
        os.makedirs(uploaded_files_dir, exist_ok=True)
        save_path = os.path.join(uploaded_files_dir, pdf_file.name)
        
        # PDF dosyasını kaydet
        with open(save_path, "wb") as f:
            f.write(pdf_file.read())
        
        
        # PDF dosyasını işle
        pdf_open = pymupdf.open(save_path)
        toc = pdf_open.get_toc()
        chunked_documents = []
                
        for i, item in enumerate(toc):
            heading = item[1]
            start_page = item[2]
            
            if i + 1 < len(toc):
                end_page = toc[i + 1][2] - 1
            else:
                end_page = pdf_open.page_count - 1  # Son başlıksa son sayfaya kadar
            
            chunk_text = ""
            for page_num in range(start_page, end_page + 1):
                chunk_text += pdf_open[page_num].get_text()
            
            chunked_documents.append(
                Document(
                    page_content=chunk_text,
                    metadata={"heading": heading, "start_page": start_page, "end_page": end_page}
                )
            )
        
        # Vektör veritabanına ekleme
        vectorstore = Chroma(persist_directory=db_name, embedding_function=embed_method)
        vectorstore.add_documents(chunked_documents)
        
        return f"{pdf_file.name} başarıyla işlendi ve vektör veritabanına eklendi!"
    except Exception as e:
        return f"Hata: {str(e)}"


In [15]:
import gradio as gr
from langchain.memory import ConversationBufferMemory
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_community.llms import Ollama
#burada get_uploaded_func yerine yukarıdaki methodu kullandık,  başarıyla işledim diyor ama uploaded_files altında pdf yok.

# Vector database name
DB_NAME = "vector-db"

# Embedding methods
embed_methods = {
    "Ollama - Nomic Embed": OllamaEmbeddings(model="nomic-embed-text")
}

def process_upload(uploaded_file, selected_embed):
    try:
        embeddings = embed_methods[selected_embed]
        save_uploaded_file(uploaded_file, embeddings, DB_NAME)
        return f"{uploaded_file.name} has been successfully added to the vector database!"
    except Exception as e:
        return f"Error: {str(e)}"

# Load vector database
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma(persist_directory=DB_NAME, embedding_function=embeddings)
retriever = vectorstore.as_retriever()

# Load LLM model
llm = Ollama(model="llama3.2")

# Chat Memory
memory = ConversationBufferMemory()

def query_rag_pipeline(user_query, history):
    # Retrieve chat history
    chat_history = memory.load_memory_variables({}).get("history", "")
    
    # Retrieve relevant documents
    retrieved_docs = vectorstore.similarity_search(user_query, k=10)
    combined_context = " ".join([doc.page_content for doc in retrieved_docs])

    # Prompt format
    prompt = f"""
    You are a telecom assistant. Your answers should be based on the context and chat history provided. If the context is not relevant to the user's query, politely state that you do not have the required information.

    Chat History: {chat_history}
    
    Context: {combined_context}
    
    Question: {user_query}
    
    Answer:
    """

    response = llm.invoke(prompt)

    # Update chat memory
    memory.save_context({"input": user_query}, {"output": response})
    
    return response

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## Server-Side RAG Assistant 🤖")
    
    with gr.Row():
        selected_embed = gr.Dropdown(choices=list(embed_methods.keys()), label="Select Embedding Method")
        file_uploader = gr.File(label="Upload a PDF")
        upload_button = gr.Button("Upload")
    
    upload_output = gr.Textbox(label="Upload Status")
    upload_button.click(process_upload, inputs=[file_uploader, selected_embed], outputs=upload_output)
    
    
    chatbot = gr.ChatInterface(fn=query_rag_pipeline, title="Chatbot Assistant", type="messages")

demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7865
Running on public URL: https://6c08e2c78f9cd633c4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
